In [1]:
### Verifier si flask est installé:
#!pip show flask

### installer flask si module absent:
# !pip install flask

In [2]:
# import requests
# import json

# base_url = "https://gutendex.com/books/?page="
# library = []
# id = 0

# for i in range(1,53):
#     response = requests.get(base_url+str(i))
#     data = json.loads(response.text)['results']
#     for d in data:
#         if 'text/plain' in d['formats'] and 'image/jpeg' in d['formats']:
#             id += 1
#             library.append(dict(id = id, title=d['title'], image=d['formats']['image/jpeg'], content=d['formats']['text/plain']))

In [3]:
import requests

base_url = "https://gutendex.com/books/"
params = {
    "mime_type": "image/jpeg",
    "page": 1,
}

library = []
id = 0

for i in range(54):
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        print("Error occurred while retrieving data.")
        break
    
    data = response.json()['results']
    if not data:
        break
    
    for book in data:
        if 'text/plain' in book['formats']:
            id += 1
            library.append({
                "id": id,
                "title": book['title'],
                "image": book['formats']['image/jpeg'],
                "content": book['formats']['text/plain']
            })
    params["page"] += 1
    
print(f"Retrieved {len(library)} books.")


Retrieved 1187 books.


In [4]:
import urllib.request
import re
from collections import defaultdict

words_per_book = []
book_id = 0

try:
    for book in library:
        url = book['content']
        with urllib.request.urlopen(url) as response:
            text = re.sub(r'[^\w\s]', '', response.read().decode('utf-8')).split("\n")

        dictionary = defaultdict(lambda: {'name': '', 'occurrence': 0, 'book': book['id']})
        book_id = book['id']
        for line in text:
            for word in line.split():
                dictionary[word.lower()]['name'] = word.lower()
                dictionary[word.lower()]['occurrence'] += 1

        words_per_book += [
            {'name': word['name'], 'occurrence': word['occurrence'], 'book': book['id']}
            for word in dictionary.values()
        ]

except Exception as e:
    print("An error occurred:", e)
    print("dernier de livre lu:",book_id)


An error occurred: <urlopen error [WinError 10060] Une tentative de connexion a échoué car le parti connecté n’a pas répondu convenablement au-delà d’une certaine durée ou une connexion établie a échoué car l’hôte de connexion n’a pas répondu>
dernier de livre lu: 728


In [5]:
import json
# save library in file (library.json) to keep it for test
with open('library.json', 'w') as file:
    json.dump(library, file)
    
# same for words_per_book
with open('words.json', 'w') as file:
    # write the data to the file in JSON format
    json.dump(words_per_book, file)

In [6]:
import json

with open('library.json', 'r') as file:
    bibliotheque = json.load(file)
    
with open('words.json', 'r') as file:
    mots_livre = json.load(file)

In [7]:
sorted_list = sorted(mots_livre, key=lambda x: x['name'])

mot = sorted_list[0]['name']
livre = []
index_table = {}

for word in sorted_list:
    if word['name'] != mot :
        #  index_table.append({'token':mot,'result':livre})
        index_table[mot] = {'token':mot,'result':livre}
        mot = word['name']
        livre = []
    livre.append({'book_ref':word['book'],'occurrence':word['occurrence']})
# index_table.append({'token':mot,'result':livre})
index_table[mot] = {'token':mot,'result':livre}

In [8]:
### AutoCompletion

def auto_completion(dict_mots, mot):
    mots_triees = sorted(dict_mots.keys())
    debut = 0
    fin = len(mots_triees) - 1
    while debut <= fin:
        milieu = (debut + fin) // 2
        if mots_triees[milieu].startswith(mot):
            break
        elif mot < mots_triees[milieu]:
            fin = milieu - 1
        else:
            debut = milieu + 1

    if debut > fin:
        return []

    mots_commencant = [mots_triees[milieu]]
    i = milieu - 1
    while i >= 0 and mots_triees[i].startswith(mot):
        mots_commencant.insert(0, mots_triees[i])
        i -= 1

    i = milieu + 1
    while i < len(mots_triees) and mots_triees[i].startswith(mot):
        mots_commencant.append(mots_triees[i])
        i += 1

    return mots_commencant

In [9]:
### Advanced Search Corrector

def levenshtein_distance(s1, s2):
    m, n = len(s1), len(s2)
    if m < n:
        return levenshtein_distance(s2, s1)
    if n == 0:
        return m
    previous_row = list(range(n + 1))
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

def wagner_fischer_distance(s1, s2, max_distance):
    m, n = len(s1), len(s2)
    if abs(m - n) > max_distance:
        return max_distance + 1
    if n == 0:
        return m
    previous_row = list(range(n + 1))
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        min_distance = max_distance + 1
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            distance = min(insertions, deletions, substitutions)
            if distance < min_distance:
                min_distance = distance
            current_row.append(distance)
        if min_distance > max_distance:
            return max_distance + 1
        previous_row = current_row
    return previous_row[-1]

def asc(target_word,max_dist):
    # Liste de mots avec une distance de Levenshtein de moins de 2 par rapport au mot recherché
    results = []
    for word in index_table:
        distance = wagner_fischer_distance(target_word, word, max_dist)
        if distance <= max_dist:
            results.append(word)
    return results

In [10]:
# Lancement de la recherche.

def find(word):
    word = word.lower()
    final = []
    max_distance = 2 #distance maximale pour l'auto correcteur
    search_result = auto_completion(index_table, word)
    if len(search_result) == 0:
        search_result = asc(word,max_distance)
    for item in search_result:
        final.append(index_table[item])
    return final     

In [ ]:
from flask import Flask, jsonify,make_response

# Créer une instance de l'application Flask
app = Flask(__name__)

app.config['JSONIFY_PRETTYPRINT_REGULAR'] = True

# Définir les routes pour l'API
@app.route('/api/v1/books/list')
def lists():
    data = bibliotheque[0:book_id]
     # Créer une réponse à partir de l'objet JSON
    response = make_response(jsonify(data))

    # Ajouter l'en-tête Content-Type à la réponse
    response.headers['Content-Type'] = 'application/json'

    return response


@app.route('/api/v1/books/search/<word>')
def search(word):
    data = find(word)
     # Créer une réponse à partir de l'objet JSON
    response = make_response(jsonify(data))

    # Ajouter l'en-tête Content-Type à la réponse
    response.headers['Content-Type'] = 'application/json'

    return response

# Démarrer le serveur de l'application Flask
if __name__ == '__main__':
    app.run(port=8000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Feb/2023 19:47:02] "GET /api/v1/books/list HTTP/1.1" 200 -
127.0.0.1 - - [22/Feb/2023 19:47:52] "GET /api/v1/books/search/sarg HTTP/1.1" 200 -
127.0.0.1 - - [22/Feb/2023 19:48:10] "GET /api/v1/books/search/sarg HTTP/1.1" 200 -
127.0.0.1 - - [22/Feb/2023 19:48:36] "GET /api/v1/books/search/sargone HTTP/1.1" 200 -
127.0.0.1 - - [22/Feb/2023 19:48:43] "GET /api/v1/books/search/sargone HTTP/1.1" 200 -
